# Read and Write Mesh Data

In [1]:
from sigmaepsilon.solid import Structure
from sigmaepsilon.solid.fem import BernoulliFrame
import numpy as np
from numpy import pi as PI


## Import Mesh from JSON-like Data

In [2]:
from sigmaepsilon.solid import BeamSection, get_section

# all units in kN and cm

L = 300.  # length of the console [cm]
F = 2.  # value of the vertical load at the free end [kN]
E = 21000.0  # Young's modulus [kN/cm3]
nu = 0.3  # Poisson's ratio [-]
fy = 23.5  # yield strength [kN/cm2]
density = 7750.0 * 1e-6  # mass density [g/cm3]
weight = density * 1e-3 * 9.81  # [kN/cm3]
alpha = 1.2e-05

d_chs = 10.0  # diameter of the tube
t_chs = 1.0  # thickness of the tube
n_chs = 32  # number of subdivisions along the perimeter when discretizing the section
section = get_section('CHS', d=d_chs, t=t_chs, n=n_chs)
beam = BeamSection(wrap=section)
beam.calculate_geometric_properties()
props = beam.geometric_properties
A = props['A']  # area
Ix = props['Ix']  # torsional inertia
Iy = props['Iy']  # second moment of inertia around the y axis
Iz = props['Iz']  # second moment of inertia around the z axis

filename = 'console_bernoulli_linstat'

In [3]:
from sigmaepsilon.io import json2dict, dict2json


def read_dict(d_in=None, jsonpath=None):
    if jsonpath is not None:
        if d_in is not None:
            dict2json(jsonpath, d_in)
        else:
            d_in = json2dict(jsonpath)
    d_out, mesh = BernoulliFrame.from_dict(d_in)
    return d_out, mesh


def solve_dict(*args, **kwargs):
    d_out, mesh = read_dict(*args, **kwargs)
    res = Structure(mesh=mesh).linsolve().nodal_dof_solution()[-1, 2, 2]
    return d_out, res


In [4]:
d_in = {
    "_version_": "0.0.1",

    "materials": {
        "Steel": {
            "E": E,
            "nu": nu,
            "fy": fy,
            "type": "steel",
            "density": density,
            "thermal-coefficient": alpha
        },
        "CustomMaterial": {
            "fy": fy,
            "density": density,
            "thermal-coefficient": alpha
        }
    },

    "sections": {
        "SteelSection": {
            "type": "CHS",
            "d": d_chs,
            "t": t_chs,
        },
        "CustomSection": {
            "A": A, "Ix": Ix, "Iy": Iy, "Iz": Iz,
        }
    },

    "frames": {
        "0": [
            [1.0, 0.0, 0.0],
            [0.0, 1.0, 0.0],
            [0.0, 0.0, 1.0]
        ]
    },

    "points": {
        "0": {
            "x": [0.0, 0.0, 0.0],
            "fixity": [True, ] * 6
        },
        "1": [L/3, 0.0, 0.0],
        "2": {
            "x": [2*L/3, 0.0, 0.0],
        },
        "3": {
            "x": [L, 0.0, 0.0],
            "mass": 1.0,
            "load": {
                "Fx": [F, 0.0, 0.0, 0.0, 0.0, 0.0],
                "Fy": [0.0, F, 0.0, 0.0, 0.0, 0.0],
                "Fz": [0.0, 0.0, F, 0.0, 0.0, 0.0]
            },
            "fixity": {
                'UX': False
            }
        }
    },

    "cells": {
        "0": {
            "nodes": [0, 1],
            "section": "SteelSection",
            "material": "Steel",
            "density": density,
            "frame": "0"
        },
        "1": {
            "nodes": [1, 2],
            "section": "SteelSection",
            "material": "Steel",
            "density": density,
        },
        "2": {
            "nodes": [2, 3],
            "section": "SteelSection",
            "material": "Steel",
            "density": density,
            "load": {
                "qx": [F, 0.0, 0.0, 0.0, 0.0, 0.0],
                "qy": [0.0, F, 0.0, 0.0, 0.0, 0.0],
                "qz": [0.0, 0.0, F, 0.0, 0.0, 0.0]
            },
        }
    },
}


In [5]:
d_out, mesh = read_dict(d_in)
res = Structure(mesh=mesh).linsolve().nodal_dof_solution()[-1, 2, 2]


In [6]:
Structure(mesh=mesh).linsolve().nodal_dof_solution().shape

(4, 6, 6)

In [7]:
d_out, uz = solve_dict(d_in)
print("Tip displacement : {}".format(uz))


Tip displacement : 2.967102148811897


In [8]:
# Bernoulli solution
EI = E * Iy
sol_exact = F * L**3 / (3 * EI)
print("Tip displacement : {}".format(sol_exact))


Tip displacement : 2.995861204893461


In [9]:
jsonpath = filename + ".json"

d_out, uz = solve_dict(d_in, jsonpath)
print("Tip displacement : {}".format(uz))

d_out, uz = solve_dict(None, jsonpath)
print("Tip displacement : {}".format(uz))


Tip displacement : 2.967102148811897
Tip displacement : 2.967102148811897


In [10]:
from sigmaepsilon.examples import download_bernoulli_console_json_linstat
from sigmaepsilon.io import json2dict
jsonpath = download_bernoulli_console_json_linstat()
d_in = json2dict(jsonpath)
d_out, mesh = BernoulliFrame.from_dict(d_in)
uz = Structure(mesh=mesh).linsolve().nodal_dof_solution()[-1, 2, 2]
print("Tip displacement : {}".format(uz))


Tip displacement : 2.967102148811897


In [ ]:
total_mass = L * A * density  # [g]
total_weight = total_mass * 1e-3 * 9.81

d_in['points']['3']['mass'] = total_mass

d_out, uz = solve_dict(d_in)
print("Tip displacement : {}".format(uz))

## Write Finite Element Mesh to Parquet files

In [11]:
d_out, mesh = read_dict(d_in)
structure = Structure(mesh=mesh)
dofsol = structure.linsolve().nodal_dof_solution(store='dofsol')
pdpath = "pd_" + filename + ".parquet"
cdpath = "cd_" + filename + ".parquet"
mesh.to_parquet(pdpath, cdpath)

## Read Finite Element Mesh from Parquet files

In [12]:
from sigmaepsilon.solid import LineMesh, PointData
from sigmaepsilon.solid.fem.cells import B2
from polymesh.space import StandardFrame
import awkward as ak

GlobalFrame = StandardFrame(dim=3)

pd = PointData(db=ak.from_parquet(pdpath))
cd = B2(db=ak.from_parquet(cdpath))
mesh = LineMesh(pd, cd, frame=GlobalFrame)
structure = Structure(mesh=mesh)
uz = pd.dofsol[-1, 2, 2]
print("Tip displacement : {}".format(uz))

Tip displacement : 2.967102148811897
